# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as mp
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2272740903992,
 'week52high': 150.7,
 'week52low': 87.62,
 'week52highSplitAdjustOnly': 146.59,
 'week52lowSplitAdjustOnly': 88.02,
 'week52change': 0.4913018372745883,
 'sharesOutstanding': 16977361187,
 'float': 0,
 'avg10Volume': 84572565,
 'avg30Volume': 78529686,
 'day200MovingAvg': 131.37,
 'day50MovingAvg': 131.4,
 'employees': 151244,
 'ttmEPS': 4.47,
 'ttmDividendRate': 0.8419932841564112,
 'dividendYield': 0.0062592609212833,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-23',
 'nextEarningsDate': '2021-07-22',
 'peRatio': 29.443743567469934,
 'beta': 1.6044473716818148,
 'maxChangePercent': 51.01884540531783,
 'year5ChangePercent': 5.371390777153147,
 'year2ChangePercent': 1.818292275091275,
 'year1ChangePercent': 0.504862513111365,
 'ytdChangePercent': 0.008977212359375,
 'month6ChangePercent': 0.014544569618878978,
 'month3ChangePercent': 0.1132507658155875,
 'month1ChangePercent': 0.05178417609834563,
 'day30ChangePercent':

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.504862513111365

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], 
            index = my_columns), 
            ignore_index = True)
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,146.98,0.7132,N/A
1,AAL,23.42,0.733014,N/A
2,AAP,201.47,0.438342,N/A
3,AAPL,136.27,0.514298,N/A
4,ABBV,116.97,0.279518,N/A
...,...,...,...,...
500,YUM,117.02,0.385206,N/A
501,ZBH,167.85,0.405408,N/A
502,ZBRA,528.04,1.065092,N/A
503,ZION,55.84,0.708722,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,73.09,4.316775,N/A
1,FCX,37.75,2.580206,N/A
2,TPR,44.96,2.371405,N/A
3,GPS,33.20,2.271383,N/A
4,DVN,30.71,1.829004,N/A
5,KSS,58.20,1.821694,N/A
6,SIVB,585.77,1.717182,N/A
7,IVZ,28.50,1.690338,N/A
8,MOS,32.17,1.689929,N/A
9,MGM,45.86,1.662079,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Benny\anaconda3\envs\AT_course\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,73.09,4.316775,2682
1,FCX,37.75,2.580206,5194
2,TPR,44.96,2.371405,4361
3,GPS,33.20,2.271383,5905
4,DVN,30.71,1.829004,6384
5,KSS,58.20,1.821694,3369
6,SIVB,585.77,1.717182,334
7,IVZ,28.50,1.690338,6879
8,MOS,32.17,1.689929,6095
9,MGM,45.86,1.662079,4275


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
#         print(symbol)
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.17,N/A,0.730194,N/A,0.255685,N/A,0.224113,N/A,0.102639,N/A,N/A
1,AAL,22.46,N/A,0.740064,N/A,0.427872,N/A,0.025739,N/A,-0.028079,N/A,N/A
2,AAP,208.07,N/A,0.434292,N/A,0.263163,N/A,0.112223,N/A,0.062405,N/A,N/A
3,AAPL,138.67,N/A,0.511826,N/A,0.014148,N/A,0.112936,N/A,0.052035,N/A,N/A
4,ABBV,120.19,N/A,0.267945,N/A,0.143261,N/A,0.130026,N/A,-0.01059,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.32,N/A,0.394668,N/A,0.100958,N/A,0.101955,N/A,-0.025586,N/A,N/A
501,ZBH,170.01,N/A,0.405205,N/A,0.103239,N/A,0.046575,N/A,-0.021045,N/A,N/A
502,ZBRA,522.83,N/A,1.071139,N/A,0.36725,N/A,0.101722,N/A,0.017659,N/A,N/A
503,ZION,55.58,N/A,0.70628,N/A,0.27883,N/A,0.0288,N/A,-0.059113,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row,f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row,f'{time_period} Price Return']=0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.740594
1      0.746535
2      0.463366
3      0.538614
4      0.245545
         ...   
500    0.413861
501    0.423762
502     0.89901
503    0.712871
504         0.4
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.669307
1       0.89901
2      0.681188
3      0.128713
4      0.413861
         ...   
500    0.312871
501    0.316832
502    0.845545
503    0.714851
504    0.455446
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.90297
1      0.265347
2      0.611881
3      0.613861
4      0.679208
         ...   
500    0.572277
501    0.348515
502    0.570297
503    0.281188
504    0.867327
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.942574
1      0.344554
2      0.853465
3      0.807921
4      0.467327
         ...   
500    0.360396
501    0.394059
502    0.673267
503    0.126733
504    0.813861
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.17,N/A,0.730194,0.740594,0.255685,0.669307,0.224113,0.90297,0.102639,0.942574,N/A
1,AAL,22.46,N/A,0.740064,0.746535,0.427872,0.89901,0.025739,0.265347,-0.028079,0.344554,N/A
2,AAP,208.07,N/A,0.434292,0.463366,0.263163,0.681188,0.112223,0.611881,0.062405,0.853465,N/A
3,AAPL,138.67,N/A,0.511826,0.538614,0.014148,0.128713,0.112936,0.613861,0.052035,0.807921,N/A
4,ABBV,120.19,N/A,0.267945,0.245545,0.143261,0.413861,0.130026,0.679208,-0.01059,0.467327,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.32,N/A,0.394668,0.413861,0.100958,0.312871,0.101955,0.572277,-0.025586,0.360396,N/A
501,ZBH,170.01,N/A,0.405205,0.423762,0.103239,0.316832,0.046575,0.348515,-0.021045,0.394059,N/A
502,ZBRA,522.83,N/A,1.071139,0.89901,0.36725,0.845545,0.101722,0.570297,0.017659,0.673267,N/A
503,ZION,55.58,N/A,0.70628,0.712871,0.27883,0.714851,0.0288,0.281188,-0.059113,0.126733,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.17,N/A,0.730194,0.740594,0.255685,0.669307,0.224113,0.90297,0.102639,0.942574,0.813861
1,AAL,22.46,N/A,0.740064,0.746535,0.427872,0.89901,0.025739,0.265347,-0.028079,0.344554,0.563861
2,AAP,208.07,N/A,0.434292,0.463366,0.263163,0.681188,0.112223,0.611881,0.062405,0.853465,0.652475
3,AAPL,138.67,N/A,0.511826,0.538614,0.014148,0.128713,0.112936,0.613861,0.052035,0.807921,0.522277
4,ABBV,120.19,N/A,0.267945,0.245545,0.143261,0.413861,0.130026,0.679208,-0.01059,0.467327,0.451485
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.32,N/A,0.394668,0.413861,0.100958,0.312871,0.101955,0.572277,-0.025586,0.360396,0.414851
501,ZBH,170.01,N/A,0.405205,0.423762,0.103239,0.316832,0.046575,0.348515,-0.021045,0.394059,0.370792
502,ZBRA,522.83,N/A,1.071139,0.89901,0.36725,0.845545,0.101722,0.570297,0.017659,0.673267,0.74703
503,ZION,55.58,N/A,0.70628,0.712871,0.27883,0.714851,0.0288,0.281188,-0.059113,0.126733,0.458911


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace =  True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop=True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,30.43,N/A,1.836481,0.992079,0.995277,0.99604,0.350767,0.988119,0.152539,0.972277,0.987129
1,MRO,14.01,N/A,1.472375,0.972277,1.121718,1.0,0.344001,0.986139,0.161522,0.976238,0.983663
2,FANG,97.97,N/A,1.379544,0.958416,1.017408,0.99802,0.273357,0.942574,0.229748,0.994059,0.973267
3,F,16.01,N/A,1.567463,0.978218,0.753744,0.986139,0.269113,0.934653,0.173476,0.984158,0.970792
4,NVDA,797.73,N/A,1.111585,0.916832,0.484349,0.942574,0.531863,1.0,0.232138,0.99604,0.963861
5,LB,72.97,N/A,4.254342,1.0,0.893361,0.992079,0.265355,0.930693,0.094394,0.924752,0.961881
6,FTNT,247.37,N/A,0.802422,0.79802,0.611982,0.972277,0.406776,0.994059,0.128561,0.966337,0.932673
7,DXC,38.49,N/A,1.581847,0.980198,0.693837,0.984158,0.468873,0.99802,0.024645,0.708911,0.917822
8,EXR,169.91,N/A,0.89009,0.841584,0.454976,0.922772,0.269336,0.936634,0.127387,0.964356,0.916337
9,DFS,126.37,N/A,1.459071,0.970297,0.393743,0.871287,0.314381,0.976238,0.053448,0.815842,0.908416


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\Benny\anaconda3\envs\AT_course\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,30.43,644,1.836481,0.992079,0.995277,0.99604,0.350767,0.988119,0.152539,0.972277,0.987129
1,MRO,14.01,1399,1.472375,0.972277,1.121718,1.0,0.344001,0.986139,0.161522,0.976238,0.983663
2,FANG,97.97,200,1.379544,0.958416,1.017408,0.99802,0.273357,0.942574,0.229748,0.994059,0.973267
3,F,16.01,1224,1.567463,0.978218,0.753744,0.986139,0.269113,0.934653,0.173476,0.984158,0.970792
4,NVDA,797.73,24,1.111585,0.916832,0.484349,0.942574,0.531863,1.0,0.232138,0.99604,0.963861
5,LB,72.97,268,4.254342,1.0,0.893361,0.992079,0.265355,0.930693,0.094394,0.924752,0.961881
6,FTNT,247.37,79,0.802422,0.79802,0.611982,0.972277,0.406776,0.994059,0.128561,0.966337,0.932673
7,DXC,38.49,509,1.581847,0.980198,0.693837,0.984158,0.468873,0.99802,0.024645,0.708911,0.917822
8,EXR,169.91,115,0.89009,0.841584,0.454976,0.922772,0.269336,0.936634,0.127387,0.964356,0.916337
9,DFS,126.37,155,1.459071,0.970297,0.393743,0.871287,0.314381,0.976238,0.053448,0.815842,0.908416


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()